# Figure 1

Eddy-fluxes, EKE, and vertically averaged zonal wind.

In [ ]:
import numpy as np
import xarray as xr
import matplotlib.pyplot as plt
from matplotlib.ticker import ScalarFormatter
import matplotlib.transforms as mtransforms
import cmocean
import numba
import scipy.stats
import os

from icon_util import regrid, eddy_flux

plt.rcParams.update({'font.size': 14})
work = os.environ['WORK']

## Load data & Eulerian mean

In [ ]:
def weighted_average(da):
    
    dp = [10000,7500,2500,7500,10000,15000,17500,15000,7500]
    dp = xr.DataArray(dp,coords=dict(plev=[100,200,250,300,400,500,700,850,1000]))
    
    mass = (dp / 9.81).sum('plev')
    integral = (da * dp / 9.81).sum('plev')
    
    return integral/mass

def theta(da):
    return (da * np.exp(np.log(1000/da['plev'])*0.2854))

In [ ]:
## reference temperature
directory = work+'/DATA/icon_simulations/atm_heldsuarez_default/3d/ta/'
levels = [10000,20000,25000,30000,40000,50000,70000,85000,100000]

ta = []

for lev in levels:
    files = [directory + '%dpa/'%lev + f for f in os.listdir(directory+'%dpa/'%lev) if f.endswith('.nc')]
    files.sort()
    ta.append(xr.open_mfdataset(files,combine='nested',concat_dim='time')['ta'])

ta = xr.concat(ta,dim='plev')


## reference zonal wind
directory = work+'/DATA/icon_simulations/atm_heldsuarez_default/3d/ua/'
levels = [10000,20000,25000,30000,40000,50000,70000,85000,100000]

ua = []

for lev in levels:
    files = [directory + '%dpa/'%lev + f for f in os.listdir(directory+'%dpa/'%lev) if f.endswith('.nc')]
    files.sort()
    ua.append(xr.open_mfdataset(files,combine='nested',concat_dim='time')['ua'])

ua = xr.concat(ua,dim='plev')

## reference meridional wind
directory = work+'/DATA/icon_simulations/atm_heldsuarez_default/3d/va/'
levels = [10000,20000,25000,30000,40000,50000,70000,85000,100000]

va = []

for lev in levels:
    files = [directory + '%dpa/'%lev + f for f in os.listdir(directory+'%dpa/'%lev) if f.endswith('.nc')]
    files.sort()
    va.append(xr.open_mfdataset(files,combine='nested',concat_dim='time')['va'])

va = xr.concat(va,dim='plev')

default = xr.Dataset(dict(ta=ta,ua=ua,va=va))

# cut spin-up
default = default.sel(time=slice(1300,501300))

default

In [ ]:
## experiment temperature
directory = work+'/DATA/icon_simulations/atm_heldsuarez_butler_exp9/3d/ta/'
levels = [10000,20000,25000,30000,40000,50000,70000,85000,100000]

ta = []

for lev in levels:
    files = [directory + '%dpa/'%lev + f for f in os.listdir(directory+'%dpa/'%lev) if f.endswith('.nc')]
    files.sort()
    ta.append(xr.open_mfdataset(files,combine='nested',concat_dim='time')['ta'])

ta = xr.concat(ta,dim='plev')



## experiment zonal wind
directory = work+'/DATA/icon_simulations/atm_heldsuarez_butler_exp9/3d/ua/'
levels = [10000,20000,25000,30000,40000,50000,70000,85000,100000]

ua = []

for lev in levels:
    files = [directory + '%dpa/'%lev + f for f in os.listdir(directory+'%dpa/'%lev) if f.endswith('.nc')]
    files.sort()
    ua.append(xr.open_mfdataset(files,combine='nested',concat_dim='time')['ua'])

ua = xr.concat(ua,dim='plev')



## experiment meridional wind
directory = work+'/DATA/icon_simulations/atm_heldsuarez_butler_exp9/3d/va/'
levels = [10000,20000,25000,30000,40000,50000,70000,85000,100000]

va = []

for lev in levels:
    files = [directory + '%dpa/'%lev + f for f in os.listdir(directory+'%dpa/'%lev) if f.endswith('.nc')]
    files.sort()
    va.append(xr.open_mfdataset(files,combine='nested',concat_dim='time')['va'])

va = xr.concat(va,dim='plev')


exp9 = xr.Dataset(dict(ta=ta,ua=ua,va=va))

exp9 = exp9.sel(time=slice(1300,501300))

exp9

In [ ]:
## experiment temperature
directory = work+'/DATA/icon_simulations/atm_heldsuarez_butler_exp4/3d/ta/'
levels = [10000,20000,25000,30000,40000,50000,70000,85000,100000]

ta = []

for lev in levels:
    files = [directory + '%dpa/'%lev + f for f in os.listdir(directory+'%dpa/'%lev) if f.endswith('.nc')]
    files.sort()
    ta.append(xr.open_mfdataset(files,combine='nested',concat_dim='time')['ta'])

ta = xr.concat(ta,dim='plev')


## experiment zonal wind
directory = work+'/DATA/icon_simulations/atm_heldsuarez_butler_exp4/3d/ua/'
levels = [10000,20000,25000,30000,40000,50000,70000,85000,100000]

ua = []

for lev in levels:
    files = [directory + '%dpa/'%lev + f for f in os.listdir(directory+'%dpa/'%lev) if f.endswith('.nc')]
    files.sort()
    ua.append(xr.open_mfdataset(files,combine='nested',concat_dim='time')['ua'])

ua = xr.concat(ua,dim='plev')


## experiment meridional wind
directory = work+'/DATA/icon_simulations/atm_heldsuarez_butler_exp4/3d/va/'
levels = [10000,20000,25000,30000,40000,50000,70000,85000,100000]

va = []

for lev in levels:
    files = [directory + '%dpa/'%lev + f for f in os.listdir(directory+'%dpa/'%lev) if f.endswith('.nc')]
    files.sort()
    va.append(xr.open_mfdataset(files,combine='nested',concat_dim='time')['va'])

va = xr.concat(va,dim='plev')


exp4 = xr.Dataset(dict(ta=ta,ua=ua,va=va))

exp4 = exp4.sel(time=slice(1300,501300))

exp4

In [ ]:
## experiment zonal wind
directory = work+'/DATA/icon_simulations/atm_heldsuarez_butler_exp8/3d/ua/'
levels = [10000,20000,25000,30000,40000,50000,70000,85000,100000]

ua = []

for lev in levels:
    files = [directory + '%dpa/'%lev + f for f in os.listdir(directory+'%dpa/'%lev) if f.endswith('.nc')]
    files.sort()
    ua.append(xr.open_mfdataset(files,combine='nested',concat_dim='time')['ua'])

ua = xr.concat(ua,dim='plev')


exp8 = xr.Dataset(dict(ua=ua))

exp8 = exp8.sel(time=slice(1300,501300))

exp8

In [ ]:
default['plev'] = default['plev'] / 100

exp9['plev'] = exp9['plev'] / 100
exp4['plev'] = exp4['plev'] / 100
exp8['plev'] = exp8['plev'] / 100

In [ ]:
# this step is ressouce intensive

T_flux_ref = eddy_flux(default['va'],default['ta']).compute()

T_mean_ref = regrid(default['ta'].mean('time').compute()).mean('longitude')

T_flux_exp9 = eddy_flux(exp9['va'],exp9['ta']).compute()

T_mean_exp9 = regrid(exp9['ta'].mean('time').compute()).mean('longitude')

T_flux_exp4 = eddy_flux(exp4['va'],exp4['ta']).compute()

T_mean_exp4 = regrid(exp4['ta'].mean('time').compute()).mean('longitude')

In [ ]:
ref = xr.Dataset(dict(T_mean=theta(T_mean_ref),T_flux=theta(T_flux_ref)))
tropical = xr.Dataset(dict(T_mean=theta(T_mean_exp9),T_flux=theta(T_flux_exp9)))
polar = xr.Dataset(dict(T_mean=theta(T_mean_exp4),T_flux=theta(T_flux_exp4)))

## Resampling and inference

In [ ]:
def rolling_wind(da):
    
    years = (da['time']/10000).astype(int)
    
    rolling = da.assign_coords(year = years).groupby('year').mean('time')
    
    rolling = rolling.sel(year=slice(1,50)).compute()
    
    zonal = regrid(rolling).mean('longitude')
    
    return zonal

In [ ]:
def rolling_eke(exp='default'):
    
    directory = work+'/wolfgang/icon_storm_track/'
    eke = xr.open_dataset(directory+'atm_heldsuarez_%s_EKE_zonal_mean_10day_highpass.nc'%(exp))['EKE'].sel(latitude=slice(0,90))
    
    # time is encoded as floating number "day as %Y%m%d.%f" with gaps
    years = (eke['time']/10000).astype(int)
    months = (eke['time']/100).astype(int)%100
    days = eke['time']%100
    
    # continous time as days since 0000-01-01
    new_time = years*360 + (months-1)* 30 + days-1
    eke['time'] = new_time
    
    # construct slices
    start = np.arange(360.25,51*360,360)
    end = np.arange(720,51*360+0.25,360)
    
    rolling = [eke.sel(time=slice(start,end)) for start, end in zip(start,end)]
    rolling = [da.mean('time').assign_coords(rolling=da.time.mean()) for da in rolling]
    
    return xr.concat(rolling,dim='rolling')

In [ ]:
@numba.guvectorize(
    "(float64[:],float64[:],float64[:,:])",
    "(n), (m) -> (m,n)",
    forceobj=True
)
def random_sample(a,nb,out):
    '''
        Draw len(nb) random samples from array a
        'ziehen mit zuruecklegen'
        
        - nb is a dummy array to get dimension size
    '''
    lt = len(a)
    variates = scipy.stats.uniform.rvs(0,lt,lt*len(nb))
    variates = variates.astype(int).reshape(len(nb),lt)
    out[:,:] = a[variates]

    
@numba.guvectorize(
    "(float64[:],float64[:],float64[:])",
    "(n), (m) -> (m)",
    forceobj=True
)    
def icdf(a,p,out):
    '''
        Inverse empirical cummulative distribution function of array at percentiles p
    '''
    sort = np.sort(a)
    out[:] = sort[np.int64(p*len(a))]


def confid(da,alpha=0.05):
    '''
        Estimate confidence intervals using the inverse empirical cummulative distribution function 
        for distrubution of bootstrap samples.
    '''
    n_bootstrap = xr.DataArray(np.arange(10000),dims=('random'))
    sample = xr.apply_ufunc(random_sample,
                         *(da,n_bootstrap),
                         input_core_dims=[['rolling'],['random']],
                         output_core_dims=[['random','rolling']],
                         dask='parallelized',
                         output_dtypes=[[np.float64]])
    
    dist = sample.mean('rolling')
    
    p = xr.DataArray([alpha/2, 1-alpha/2],dims=('percentile'))
    values = xr.apply_ufunc(icdf,
                          *(dist,p),
                          input_core_dims=[['random'],['percentile']],
                          output_core_dims=[['percentile']],
                          dask='parallelized',
                          output_dtypes=[[dist.dtype]])
    values['percentile'] = p
    
    return values

In [ ]:
# resample zonal mean zonal wind
zonal_ref = rolling_wind(default['ua'])
zonal_exp4 = rolling_wind(exp4['ua'])
zonal_exp8 = rolling_wind(exp8['ua'])
zonal_exp9 = rolling_wind(exp9['ua'])

In [ ]:
# mass weighted vertical average
barotropic_ref = weighted_average(zonal_ref)
barotropic_exp4 = weighted_average(zonal_exp4)
barotropic_exp8 = weighted_average(zonal_exp8)
barotropic_exp9 = weighted_average(zonal_exp9)

# load and resample eke
eke_ref = rolling_eke('default')
eke_exp4 = rolling_eke('butler_exp4')
eke_exp8 = rolling_eke('butler_exp8')
eke_exp9 = rolling_eke('butler_exp9')

In [ ]:
# CI estimation takes 1 minute

eke_ref_CI = confid(eke_ref)
eke_exp4_CI = confid(eke_exp4)
eke_exp8_CI = confid(eke_exp8)
eke_exp9_CI = confid(eke_exp9)

barotropic_ref_CI = confid(barotropic_ref.rename(year='rolling'))
barotropic_exp4_CI = confid(barotropic_exp4.rename(year='rolling'))
barotropic_exp8_CI = confid(barotropic_exp8.rename(year='rolling'))
barotropic_exp9_CI = confid(barotropic_exp9.rename(year='rolling'))

## Plotting

In [ ]:
 # plotting

fig = plt.figure(figsize=(8,8))

# temperature & eddy fluxes

ax1 = fig.add_subplot(3,2,1)

C0 = (ref['T_mean']).plot(ax=ax1,x='latitude',cmap=cmocean.cm.thermal,levels=np.arange(270,355,5),extend='both',add_colorbar=False)
(ref['T_flux']).plot.contour(ax=ax1,x='latitude',colors='k',levels=np.arange(3,40,3),linestyle='-')
(ref['T_flux']).plot.contour(ax=ax1,x='latitude',colors='k',levels=np.arange(-40,0,3),linestyle='--')


ax2 = fig.add_subplot(3,2,3)

(tropical['T_mean']-ref['T_mean']).plot(ax=ax2,x='latitude',cmap=cmocean.cm.balance,levels=np.arange(-8,8.5,0.5),extend='both',add_colorbar=False)
(tropical['T_flux']-ref['T_flux']).plot.contour(ax=ax2,x='latitude',colors='k',levels=np.arange(1,20,1),linestyle='-')
(tropical['T_flux']-ref['T_flux']).plot.contour(ax=ax2,x='latitude',colors='k',levels=np.arange(-20,0,1),linestyle='-')


ax3 = fig.add_subplot(3,2,5)

C2 = (polar['T_mean']-ref['T_mean']).plot(ax=ax3,x='latitude',cmap=cmocean.cm.balance,levels=np.arange(-8,8.5,0.5),extend='both',add_colorbar=False)
(polar['T_flux']-ref['T_flux']).plot.contour(ax=ax3,x='latitude',colors='k',levels=np.arange(1,20,1),linestyle='-')
(polar['T_flux']-ref['T_flux']).plot.contour(ax=ax3,x='latitude',colors='k',levels=np.arange(-20,0,1),linestyle='-')


# eddy-kinetic energy

ax4 = fig.add_subplot(2,2,2)

l1 = (eke_ref.mean('rolling')/1000).plot(ax=ax4,linestyle='-')
l2 = (eke_exp9.mean('rolling')/1000).plot(ax=ax4,linestyle='--')
l3 = (eke_exp4.mean('rolling')/1000).plot(ax=ax4,linestyle=':')
l4 = (eke_exp8.mean('rolling')/1000).plot(ax=ax4,linestyle='-.')


ax4.fill_between(eke_ref['latitude'].values,eke_ref_CI.isel(percentile=0).values/1000,eke_ref_CI.isel(percentile=1).values/1000,alpha=0.3)
ax4.fill_between(eke_exp9['latitude'].values,eke_exp9_CI.isel(percentile=0).values/1000,eke_exp9_CI.isel(percentile=1).values/1000,alpha=0.3)
ax4.fill_between(eke_exp4['latitude'].values,eke_exp4_CI.isel(percentile=0).values/1000,eke_exp4_CI.isel(percentile=1).values/1000,alpha=0.3)
ax4.fill_between(eke_exp8['latitude'].values,eke_exp8_CI.isel(percentile=0).values/1000,eke_exp8_CI.isel(percentile=1).values/1000,alpha=0.3)


ax4.set_yscale('linear')
ax4.set_ylim([0,800])
ax4.set_xlim([0,90])
ax4.set_xticks([0,30,60,90])
ax4.set_ylabel(r'Eddy-kinetic energy [kJ m$^{-2}$]',labelpad=10)
ax4.set_xlabel('Latitude [°N]')
ax4.set_xticks([15,45,75],minor=True)
ax4.grid(which='both')
ax4.yaxis.set_label_position("right")
ax4.yaxis.tick_right()
ax4.tick_params(axis='y',left=True,right=True)

ax4.legend([*l1,*l2,*l3,*l4],['Reference','Tropical warming','Arctic warming','Combined forcing'],fontsize=10,loc='upper right')


# barotropic jet

ax5 = fig.add_subplot(2,2,4)

l1 = (barotropic_ref.mean('year')).plot(ax=ax5,linestyle='-')
l2 = (barotropic_exp9.mean('year')).plot(ax=ax5,linestyle='--')
l3 = (barotropic_exp4.mean('year')).plot(ax=ax5,linestyle=':')
l4 = (barotropic_exp8.mean('year')).plot(ax=ax5,linestyle='-.')

ax5.fill_between(barotropic_ref['latitude'].values,barotropic_ref_CI.isel(percentile=0).values,barotropic_ref_CI.isel(percentile=1).values,alpha=0.3)
ax5.fill_between(barotropic_exp9['latitude'].values,barotropic_exp9_CI.isel(percentile=0).values,barotropic_exp9_CI.isel(percentile=1).values,alpha=0.3)
ax5.fill_between(barotropic_exp4['latitude'].values,barotropic_exp4_CI.isel(percentile=0).values,barotropic_exp4_CI.isel(percentile=1).values,alpha=0.3)
ax5.fill_between(barotropic_exp8['latitude'].values,barotropic_exp8_CI.isel(percentile=0).values,barotropic_exp8_CI.isel(percentile=1).values,alpha=0.3)

ax5.set_yscale('linear')
ax5.set_ylim([-5,20])
ax5.set_xlim([0,90])
ax5.set_xticks([0,30,60,90])
ax5.set_ylabel(r'Vertically averaged zonal wind [m s$^{-1}$]',labelpad=20)
ax5.set_xlabel('Latitude [°N]')
ax5.set_xticks([15,45,75],minor=True)
ax5.grid(which='both')
ax5.yaxis.set_label_position("right")
ax5.yaxis.tick_right()
ax5.tick_params(axis='y',left=True,right=True)



# configure axes

for ax in [ax1,ax2,ax3]:
    
    ax.set_xlabel('')
    ax.set_ylim(1000,100)
    ax.set_yscale('log')
    ax.set_xticks([-60,0,60],minor=False)
    ax.set_xticks([-90,-30,30,90],minor=True)
    ax.set_yticks([250,500],minor=True)
    
    ax.set_ylabel('Pressure [hPa]')
    ax.yaxis.set_major_formatter(ScalarFormatter())
    ax.yaxis.set_minor_formatter(ScalarFormatter())
    

ax3.set_xlabel('Latitude [°N]')

ax1.set_title('Reference',weight='bold',fontsize='smaller')

ax2.set_title('Tropical warming',weight='bold',fontsize='smaller')

ax3.set_title('Arctic warming',weight='bold',fontsize='smaller')
    

fig.subplots_adjust(0,0,1,1,0.25,0.25)


cbar0 = plt.colorbar(C0,ax=ax1,orientation='vertical',aspect=15)
cbar0.set_label(r'Potential temperature [K]',fontsize='medium')
cbar0.ax.set_yticks(cbar0.ax.get_yticks()[::2])

cbar2 = plt.colorbar(C2,ax=[ax2,ax3],orientation='vertical',aspect=30)
cbar2.set_label(r'Potential temperature response [K]',fontsize='medium')


trans = mtransforms.ScaledTranslation(-45/72, -20/72, fig.dpi_scale_trans)

ax1.text(-0.1,1.05,'a)',transform=ax1.transAxes+trans,fontsize='large',va='bottom')
ax2.text(-0.1,1.05,'b)',transform=ax2.transAxes+trans,fontsize='large',va='bottom')
ax3.text(-0.1,1.05,'c)',transform=ax3.transAxes+trans,fontsize='large',va='bottom')
ax4.text(0.07,1.05,'d)',transform=ax4.transAxes+trans,fontsize='large',va='bottom')
ax5.text(0.07,1.05,'e)',transform=ax5.transAxes+trans,fontsize='large',va='bottom')
